In [ ]:
import numpy as np
import sympy as sp
import minterpy as mp
import minterpy_levelsets as ls

This is a benchmark of fitting errors using GPLS method for algebraic varieties.

GPLS method can be used to find an implicit representation of pointcloud surfaces. First, we show how to generate points on algebraic varieties. In order to generate pointcloud on surfaces, we use the implicit equation and sample points on the zero levelset.

## Random sampling points on implicit surfaces

This demonstrates sampling of random points on implicit surfaces represented by algebraic varieties.

### Step 1: Initialize the implicit polynomial as a sympy expression

In [ ]:
x, y, z = sp.symbols('x y z')

#### Run any one of the cells in this step

In [ ]:
## Ellipsoid
a = 0.8
b = 0.9
c = 1.0
expr = (x**2/a**2) + (y**2/b**2) + (z**2/c**2) - 1

In [ ]:
## Biconcave disc
d = 0.5
c = 0.375
expr = (d**2 + x**2 + y**2 + z**2)**3 - 8*d**2 * (y**2 + z**2) - c**4

In [ ]:
## Torus
R = 0.5
r = 0.3
expr = (x**2 + y**2 + z**2 + R**2 - r**2)**2 - 4*R**2*(x**2 + y**2)

In [ ]:
## double Torus
expr = ((x**2+y**2)**2-x**2+y**2)**2+z**2-0.01

In [ ]:
## Genus-2 surface
scale = 1.8
expr = 2*(y*scale)*((y*scale)**2 - 3*(x*scale)**2)*(1-(z*scale)**2) + ((x*scale)**2 + (y*scale)**2)**2 - (9*(z*scale)**2 - 1)*(1 - (z*scale)**2)

In [ ]:
## Klein surface
expr = (x**2 + y**2 + z**2 + 2*y - 1)*((x**2 +  y**2 + z**2 - 2*y - 1)**2 - 8*z**2) + 16*x*z*(x**2 + y**2 + z**2  - 2*y - 1)                

### Step 2: Convert sympy polynomial to minterpy polynomial

First parameter is the SymPy Poly object. Second parameter specifies the target basis for the polynomial representation in minterpy. By default, it is CanonicalPoly.

In [ ]:
poly = sp.Poly(expr, x, y, z)
newt_poly_exact = ls.sympy_to_mp(poly, mp.NewtonPolynomial)

### Step 3: Sample points on the zero isocontour

In [ ]:
point_data = ls.sample_points(newt_poly_exact,  # Polynomial in Newton basis
                              200,        # Number of points to be sampled
                              bounds=4.0, # Boundary of the Cubic domain to be sampled
                              tol=1e-15,  # Tolerance in solution
                              random_seed=42) # random seed

### (Optional) Step 4: Visualize pointcloud and exact surface

#### Pointcloud as Paraview VTK file

In [ ]:
ls.output_VTK(point_data)

#### Exact surface as Paraview VTR file

In [ ]:
ls.output_VTR(newt_poly_exact, bounds=1.0)

###  Perform surface fitting

In [ ]:
poly = ls.LevelsetPoly(point_data, method='BK', tol=1e-11, verbose=True)

### Estimate error of fit

In [ ]:
distance_errors = poly(point_data) / np.linalg.norm(poly.compute_gradients_at(point_data),axis=1)
linf_error = np.max(np.abs(distance_errors))
print(f"L_inf error = {linf_error}")

### Compare coefficients in Newton basis

In [ ]:
np.max(np.abs(poly.newton_coeffs/poly.newton_coeffs[0] - newt_poly_exact.coeffs/newt_poly_exact.coeffs[0]))

### Validation error

In [ ]:
validation_point_data = ls.sample_points(newt_poly_exact,  # Polynomial in Newton basis
                              100,        # Number of points to be sampled
                              bounds=4.0, # Boundary of the Cubic domain to be sampled
                              tol=1e-15,  # Tolerance in solution
                              random_seed=1729) # random seed

In [ ]:
validation_distance_errors = poly(validation_point_data) / np.linalg.norm(poly.compute_gradients_at(validation_point_data),axis=1)
linf_validation_error = np.max(np.abs(validation_distance_errors))
print(f"L_inf error = {linf_validation_error:.3}")